In [101]:
import pandas as pd
import numpy as np
from scipy import stats


In [102]:
HealtyData = pd.read_table('ucec-rsem-fpkm-tcga_paired.txt',
                     delimiter='\t')
HealtyData.iloc[:,:]

,Hugo_Symbol,Entrez_Gene_Id,TCGA-BK-A4ZD,TCGA-AJ-A3NC,TCGA-AJ-A3NE,TCGA-BG-A3PP,TCGA-AJ-A3NH,TCGA-BG-A3EW
0,PDGFD,80310,141.02,409.15,265.87,1447.15,223.41,193.01
1,SHANK3,85358,511.00,665.29,447.82,36.53,177.53,379.04
2,FABP3,2170,3491.39,860.08,674.59,866.07,884.29,3372.43
3,TSPAN4,7106,1832.01,3303.00,1674.06,3103.19,1216.75,4388.98
4,HIST1H2BJ,8970,37.32,18.56,59.55,9.56,100.13,57.89
...,...,...,...,...,...,...,...,...
19311,SERPIND1,3053,0.62,5.19,125.24,105.89,7.34,15.56
19312,AL162431.1,0,0.00,0.00,0.00,0.00,0.00,0.00
19313,DDX1,1653,1508.65,1331.57,1175.27,1519.15,1447.15,1407.55
19314,OR52I1,390037,0.00,0.00,0.00,0.00,0.00,0.00


In [103]:
diseasedData = pd.read_table('ucec-rsem-fpkm-tcga-t_paired.txt',
                     delimiter='\t')
#diseasedData.tail(10)

In [97]:

def FilterData(DataFrame1,DataFrame2):
    DataRejected=[];
    returnedFilteredData=pd.merge(DataFrame1,DataFrame2,on='Hugo_Symbol').drop(columns=['Entrez_Gene_Id_y','Entrez_Gene_Id_x'])
    # here FiftyPercentZeros=6 which is 50% of the Healthy and diseased Data as total number of columns =12
    FiftyPercentZeros=(DataFrame1.shape[1]-2) 
    for i in range (returnedFilteredData.shape[0]):
        SumOfZeros=0;
        for j in range (returnedFilteredData.shape[1]-1):
            if(returnedFilteredData.iloc[i,j]==0.0):
                SumOfZeros+=1;
        if(SumOfZeros >= FiftyPercentZeros):
            DataRejected.append(returnedFilteredData.iloc[i,0])
    print('DataRejected',len(DataRejected))
    updatedData=returnedFilteredData
    #Delete the Genes
    for i in range (len(DataRejected)):
        returnedFilteredData= returnedFilteredData.drop(returnedFilteredData[returnedFilteredData['Hugo_Symbol']==DataRejected[i]].index.values)   
    returnedHealtyData=returnedFilteredData.iloc[:,:7]
    returnedDiseasedData=returnedFilteredData.iloc[:,7:]
    returnedDiseasedData.insert(0,'Hugo_Symbol',returnedFilteredData.iloc[:,0],True)
    return returnedHealtyData,returnedDiseasedData


In [98]:
rh,rd=FilterData(HealtyData,diseasedData)
rh

DataRejected 1812


,Hugo_Symbol,TCGA-BK-A4ZD_x,TCGA-AJ-A3NC_x,TCGA-AJ-A3NE_x,TCGA-BG-A3PP_x,TCGA-AJ-A3NH_x,TCGA-BG-A3EW_x
0,PDGFD,141.02,409.15,265.87,1447.15,223.41,193.01
1,SHANK3,511.00,665.29,447.82,36.53,177.53,379.04
2,FABP3,3491.39,860.08,674.59,866.07,884.29,3372.43
3,TSPAN4,1832.01,3303.00,1674.06,3103.19,1216.75,4388.98
4,HIST1H2BJ,37.32,18.56,59.55,9.56,100.13,57.89
...,...,...,...,...,...,...,...
19309,SP4,72.01,96.68,86.43,182.55,92.05,59.13
19310,CLEC4D,9.48,2.18,8.99,0.00,5.82,4.82
19311,SERPIND1,0.62,5.19,125.24,105.89,7.34,15.56
19313,DDX1,1508.65,1331.57,1175.27,1519.15,1447.15,1407.55


In [99]:
rd

,Hugo_Symbol,TCGA-BK-A4ZD_y,TCGA-AJ-A3NC_y,TCGA-AJ-A3NE_y,TCGA-BG-A3PP_y,TCGA-AJ-A3NH_y,TCGA-BG-A3EW_y
0,PDGFD,20.71,255.00,44.25,61.25,97.36,52.08
1,SHANK3,25.35,78.89,63.45,36.27,100.83,81.71
2,FABP3,82.87,193.01,85.82,269.60,164.42,141.02
3,TSPAN4,351.14,361.04,379.04,493.56,902.89,486.75
4,HIST1H2BJ,23.25,152.28,103.69,37.85,144.01,105.89
...,...,...,...,...,...,...,...
19309,SP4,79.45,65.26,48.52,71.00,133.36,69.52
19310,CLEC4D,7.34,9.06,5.63,1.81,8.65,2.56
19311,SERPIND1,0.00,22.43,0.00,2.25,0.32,0.89
19313,DDX1,1674.06,1487.87,2240.11,1487.87,2133.97,1769.57


In [14]:
#WholeData.shape[1]

In [ ]:
#WholeData.loc[1067656][3]

In [ ]:
HealtyData.iloc[0,2:].to_numpy()


In [ ]:
HealtyData.iloc[17,2:].to_numpy()

In [ ]:
r,p=stats.pearsonr(HealtyData.iloc[17,2:].to_numpy(), diseasedData.iloc[17,2:].to_numpy())
r

In [104]:
#Correlation Function that retrurns the Pearson’s correlation coefficient and store it in the last column of a Dataframe  
CC_DF=HealtyData[['Hugo_Symbol','Entrez_Gene_Id']].copy() #Dataframe that has Names of Genes and their ids
CC_DF['Correlation']=""
def GetCorrelation(normalData,canceredData):
    for i in range (normalData.shape[0]):
        r,p=stats.pearsonr(normalData.iloc[i,2:].to_numpy(), canceredData.iloc[i,2:].to_numpy())
        CC_DF.iloc[i,2]=r
    return CC_DF
        


In [105]:
#function that Sort the Correlation values 


def SortCorrelation(correlationValues,ColumnName):
    HighCC=[]
    LowestCC=[]
    SortedDataAscending=correlationValues.sort_values(by=[ColumnName])
    SortedDataDescending=correlationValues.sort_values(ascending=False,by=[ColumnName])
    CCMax=SortedDataDescending.iloc[0,2]
    CCMin=SortedDataAscending.iloc[0,2]
    for i in range (SortedDataDescending.shape[0]): #SortedDataDescending.shape[1]==SortedDataAscending.shape[1]
        if(SortedDataDescending.iloc[i,2]==1):
            HighCC.append([SortedDataDescending.iloc[i,0],SortedDataDescending.iloc[i,1],SortedDataDescending.iloc[i,2]])
    for i in range(SortedDataAscending.shape[0]):
        if(SortedDataAscending.iloc[i,2]==CCMin):
            LowestCC.append([SortedDataAscending.iloc[i,0],SortedDataAscending.iloc[i,1],SortedDataAscending.iloc[i,2]])
    return SortedDataDescending,SortedDataAscending,HighCC,LowestCC

#return max and minimum correlated data
#def GetMaxandMincorrelate(CorrelationValues):
    
    

In [106]:
CorrealtedData=GetCorrelation(HealtyData,diseasedData)
#CorrealtedData.sort_values(ascending=False,by=['Correlation']).head(29)

C:\Users\Dell\anaconda3\lib\site-packages\scipy\stats\stats.py:3845: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())


In [107]:
SortedValues=CorrealtedData.sort_values(ascending=False,by=['Correlation'])
for i in range (SortedValues.shape[0]):
    if(SortedValues.iloc[i,2]==1):
        print(" ")
        print(SortedValues.iloc[i,:])
   

 
Hugo_Symbol       FOXE1
Entrez_Gene_Id     2304
Correlation           1
Name: 16490, dtype: object
 
Hugo_Symbol       OR3A1
Entrez_Gene_Id     4994
Correlation           1
Name: 4570, dtype: object
 
Hugo_Symbol       UGT2B10
Entrez_Gene_Id       7365
Correlation             1
Name: 3943, dtype: object
 
Hugo_Symbol       ANKRD30A
Entrez_Gene_Id       91074
Correlation              1
Name: 13269, dtype: object
 
Hugo_Symbol        MTRNR2L1
Entrez_Gene_Id    100462977
Correlation               1
Name: 649, dtype: object
 
Hugo_Symbol       SULT6B1
Entrez_Gene_Id     391365
Correlation             1
Name: 6954, dtype: object
 
Hugo_Symbol       TMEM244
Entrez_Gene_Id     253582
Correlation             1
Name: 18767, dtype: object
 
Hugo_Symbol         RLN3
Entrez_Gene_Id    117579
Correlation            1
Name: 9333, dtype: object
 
Hugo_Symbol       CCDC83
Entrez_Gene_Id    220047
Correlation            1
Name: 16364, dtype: object
 
Hugo_Symbol         CIB3
Entrez_Gene_Id    117286


In [108]:
SortedCorrelationDesc,SortedCorrelationAscen,Highesttt,Lowest=SortCorrelation(CorrealtedData,'Correlation')

SortedCorrelationDesc


,Hugo_Symbol,Entrez_Gene_Id,Correlation
16490,FOXE1,2304,1
4570,OR3A1,4994,1
3943,UGT2B10,7365,1
13269,ANKRD30A,91074,1
649,MTRNR2L1,100462977,1
...,...,...,...
19224,SOX1,6656,NaN
19264,SSX4,6759,NaN
19295,PRB1,5542,NaN
19312,AL162431.1,0,NaN


In [109]:
SortedCorrelationAscen

,Hugo_Symbol,Entrez_Gene_Id,Correlation
4719,PPP2R4,5524,-0.993727
1242,AK4,205,-0.983739
12837,SUSD1,64420,-0.98056
14164,AIF1L,83543,-0.972803
5444,PHLPP2,23035,-0.970304
...,...,...,...
19224,SOX1,6656,NaN
19264,SSX4,6759,NaN
19295,PRB1,5542,NaN
19312,AL162431.1,0,NaN


In [110]:
Highesttt

[['FOXE1', 2304, 1.0],
 ['OR3A1', 4994, 1.0],
 ['UGT2B10', 7365, 1.0],
 ['ANKRD30A', 91074, 1.0],
 ['MTRNR2L1', 100462977, 1.0],
 ['SULT6B1', 391365, 1.0],
 ['TMEM244', 253582, 1.0],
 ['RLN3', 117579, 1.0],
 ['CCDC83', 220047, 1.0],
 ['CIB3', 117286, 1.0],
 ['CFC1B', 55997, 1.0],
 ['LCE5A', 254910, 1.0],
 ['RP1-228P16.5', 0, 1.0],
 ['AL355490.1', 0, 1.0],
 ['ATP6C', 0, 1.0],
 ['RP11-761B3.1', 0, 1.0],
 ['VRTN', 55237, 1.0],
 ['MRGPRG', 386746, 1.0],
 ['HAO1', 54363, 1.0],
 ['LALBA', 3906, 1.0],
 ['AP001888.1', 0, 1.0],
 ['RP11-512M8.5', 0, 1.0]]

In [111]:
Lowest

[['PPP2R4', 5524, -0.9937271337541674]]

In [112]:
HighestCCdf=pd.DataFrame (Highesttt,columns=['Hugo_Symbol','Entrez_Gene_Id','Correlation'])
LowestCCdf=pd.DataFrame (Lowest,columns=['Hugo_Symbol','Entrez_Gene_Id','Correlation'])
def getGenesExpressionByName(Highest_CC_DF,Lowest_CC_DF):
    returnHighestHealthyData=pd.merge(Highest_CC_DF,HealtyData,on='Hugo_Symbol').drop(columns=['Entrez_Gene_Id_y'])
    retrunHighestDiseasedData=pd.merge(Highest_CC_DF,diseasedData,on='Hugo_Symbol').drop(columns=['Entrez_Gene_Id_y'])
    returnLowestHealthyData=pd.merge(Lowest_CC_DF,HealtyData,on='Hugo_Symbol').drop(columns=['Entrez_Gene_Id_y'])
    retrunLowestDiseasedData=pd.merge(Lowest_CC_DF,diseasedData,on='Hugo_Symbol').drop(columns=['Entrez_Gene_Id_y'])
    return returnHighestHealthyData,retrunHighestDiseasedData,returnLowestHealthyData,retrunLowestDiseasedData
        


In [113]:
a,b,c,d=getGenesExpressionByName(HighestCCdf,LowestCCdf)
c

,Hugo_Symbol,Entrez_Gene_Id_x,Correlation,TCGA-BK-A4ZD,TCGA-AJ-A3NC,TCGA-AJ-A3NE,TCGA-BG-A3PP,TCGA-AJ-A3NH,TCGA-BG-A3EW
0,PPP2R4,5524,-0.993727,3280.18,2148.82,2573.36,2451.44,3168.41,2627.46


In [114]:
d

,Hugo_Symbol,Entrez_Gene_Id_x,Correlation,TCGA-BK-A4ZD,TCGA-AJ-A3NC,TCGA-AJ-A3NE,TCGA-BG-A3PP,TCGA-AJ-A3NH,TCGA-BG-A3EW
0,PPP2R4,5524,-0.993727,1407.55,4181.07,3190.46,3564.78,1923.14,2895.31


In [49]:
# import pandas library 
import pandas as pd 

# dictionary with list object in values 
details = { 
	'Name' : ['Ankit', 'Aishwarya', 'Shaurya', 'Shivangi'], 
	'Age' : [23, 21, 22, 21], 
	'University' : ['BHU', 'JNU', 'DU', 'BHU'], 
} 

# creating a Dataframe object 
df = pd.DataFrame(details, columns = ['Name', 'Age', 'University'], 
				index = ['a', 'b', 'c', 'd']) 

# return a new dataframe by dropping a 
# row 'c' from dataframe 
update_df=df;
update_df = update_df.drop(update_df[update_df['Name']=='Ankit'].index.values) 
update_df = update_df.drop(update_df[update_df['Name']=='Aishwarya'].index.values) 
update_df 
#print()

,Name,Age,University
c,Shaurya,22,DU
d,Shivangi,21,BHU
